In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict
import numpy as np
from geopy.distance import geodesic

In [ ]:
df = pd.read_csv("Spire-AISDataSample-20240914-PortDampier.csv")

In [ ]:
def haversine_vectorized(lonlat1, lonlat2):
    """
    Calculate the great circle distance between two points
    on the earth specified in decimal degrees.
    """
    # Convert decimal degrees to radians
    lat1, lon1 = np.radians(lonlat1[:, 0]), np.radians(lonlat1[:, 1])
    lat2, lon2 = np.radians(lonlat2[:, 0]), np.radians(lonlat2[:, 1])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371.0088 * c  # Earth radius in kilometers
    return km

In [ ]:
def check_proximity_with_time_vectorized(df, min_distance=0.05, min_periods=12, time_bin_input = '10min'):
    print("Starting proximity check...")

    df['position_timestamp'] = pd.to_datetime(df['position_timestamp'])
    df.sort_values(by='position_timestamp', inplace=True)
    df.reset_index(drop=True, inplace=True)

    time_bin_size = time_bin_input
    df['time_bin'] = df['position_timestamp'].dt.floor(time_bin_size)
    time_groups = df.groupby('time_bin')

    min_time = df['time_bin'].min()
    max_time = df['time_bin'].max()
    all_time_bins = pd.date_range(start=min_time, end=max_time, freq=time_bin_size)

    pair_counters = {}
    proximity_events = defaultdict(int)
    relevant_positions = []
    proximity_id = 0
    proximity_event_map = {}

    for i, time_bin in enumerate(all_time_bins):
        if time_bin not in time_groups.groups:
            continue

        group = time_groups.get_group(time_bin)
        if len(group['mmsi'].unique()) < 2:
            continue

        vessel_df = group.groupby('mmsi').first().reset_index()
        if len(vessel_df['mmsi'].unique()) < 2:
            continue

        latitudes = vessel_df['latitude'].values
        longitudes = vessel_df['longitude'].values
        positions = np.column_stack((latitudes, longitudes))
        mmsis = vessel_df['mmsi'].values

        vessel_indices = np.arange(len(positions))
        vessel_pairs = list(combinations(vessel_indices, 2))

        if not vessel_pairs:
            continue

        indices1 = np.array([i for i, j in vessel_pairs])
        indices2 = np.array([j for i, j in vessel_pairs])

        pos1 = positions[indices1]
        pos2 = positions[indices2]
        mmsi1 = mmsis[indices1]
        mmsi2 = mmsis[indices2]

        distances = haversine_vectorized(pos1, pos2)
        close_indices = np.where(distances < min_distance)[0]

        close_pairs = [tuple(sorted((mmsi1[idx], mmsi2[idx]))) for idx in close_indices]

        for idx in close_indices:
            m1, m2 = mmsi1[idx], mmsi2[idx]
            pair = tuple(sorted((m1, m2)))

            # Assign a new proximity event ID if the pair is not already in the map
            if pair not in proximity_event_map:
                proximity_event_map[pair] = proximity_id
                proximity_id += 1

            # Use the assigned proximity event ID
            current_proximity_id = proximity_event_map[pair]

            # Collect relevant positions in the list
            lat1, lon1 = pos1[idx]
            lat2, lon2 = pos2[idx]
            timestamp = time_bin

            relevant_positions.append({
                'proximity_id': current_proximity_id,
                'mmsi1': m1,
                'mmsi2': m2,
                'timestamp': timestamp,
                'lat1': lat1,
                'lon1': lon1,
                'lat2': lat2,
                'lon2': lon2
            })

        if i == 0:
            for pair in close_pairs:
                pair_counters[pair] = 1
        else:
            current_bin_pairs = defaultdict(int)
            for pair in list(pair_counters.keys()):
                if pair in close_pairs:
                    pair_counters[pair] += 1
                    close_pairs.remove(pair)
                    if pair_counters[pair] >= min_periods:
                        proximity_events[pair] += 1
                        del pair_counters[pair]
                else:
                    del pair_counters[pair]

            for pair in close_pairs:
                pair_counters[pair] = 1

    # Convert the list of relevant positions to a DataFrame
    relevant_positions_df = pd.DataFrame(relevant_positions)

    # Sort the proximity_events dictionary by the event count in descending order
    sorted_proximity_events = dict(sorted(proximity_events.items(), key=lambda x: x[1], reverse=True))

    return pair_counters, sorted_proximity_events, relevant_positions_df


In [ ]:
pair_counters, proximity_events, rp = check_proximity_with_time_vectorized(df)

proximity_events


Starting proximity check...


{(357530000, 636023069): 11}

In [ ]:
rp

,proximity_id,mmsi1,mmsi2,timestamp,lat1,lon1,lat2,lon2
0,0,357530000,636023069,2024-09-14 00:00:00+00:00,-20.406177,116.480107,-20.406442,116.480005
1,0,357530000,636023069,2024-09-14 00:10:00+00:00,-20.406350,116.480040,-20.406613,116.479962
2,0,357530000,636023069,2024-09-14 00:20:00+00:00,-20.406557,116.480027,-20.406803,116.479980
3,0,357530000,636023069,2024-09-14 00:30:00+00:00,-20.406552,116.479987,-20.406818,116.479990
4,0,357530000,636023069,2024-09-14 00:40:00+00:00,-20.406462,116.480080,-20.406730,116.480018
...,...,...,...,...,...,...,...,...
143,0,357530000,636023069,2024-09-14 23:10:00+00:00,-20.405307,116.481120,-20.405490,116.480902
144,0,357530000,636023069,2024-09-14 23:20:00+00:00,-20.405367,116.481067,-20.405590,116.480882
145,0,357530000,636023069,2024-09-14 23:30:00+00:00,-20.405525,116.480560,-20.405738,116.480350
146,0,357530000,636023069,2024-09-14 23:40:00+00:00,-20.406037,116.480213,-20.406302,116.480062
